# RES-STRAT
Reservoir stratification model<br>
Original code in FORTRAN 90 available on __[GihHub](https://github.com/IMMM-SFA/res-strat)__<br>

### NOTE
- Output files are stored in CSV format under the folder "data" in the home directory of your JupyterHub account
- Most of the optimization rotates around removing redundant loops and ifs. The original code is preserved as comment, to be used as reference.
- The Geometry of the reservoir is stored in memory as a Pandas dataframe to speed up access to specific records
- Functions __LayerChar_d__ and __LayerChar_v__ work on the Geometry dataframe (previous point) and were added to replace a number of simila sections of code that are meant to characterize the layers based on the geometry of the reservoir
- The code is still verey slow when compared with the original FORTRAN. There is still a lot of room for improvement, but the key here is to be able to understand the logic and (hopefully) reduce the complexity...

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import os
from math import *

In [ ]:
# the following functions imitate the Fortran index lower bound (1 instead of 0) for both

# array declarations (adds an extra dummy element at index 0)
def dimArray(dim,fill=0):
    a=np.empty(dim + 1)
    a.fill(fill)
    return a
# range for loops (returns a range that includes both limits)
def range1(start,end,step=1):
    out=None
    if step == 1:
        out=range(start,end+1)
    elif step == -1:
        out=range(start,end-1,step)
    return out
# converts the sum function to nansum (excludes NaN values) 
def sum(array):
    return np.nansum(array)

# converts the min function to excludes NaN values
def min(a,b):
    if isnan(a) and isnan(b):
        return np.nan
    if isnan(a):
        a = 9e99
    if isnan(b):
        b = 9e99
    if a < b:
        return a
    else:
        return b

# converts the max function to excludes NaN values
def max(a,b):
    if isnan(a) and isnan(b):
        return np.nan
    if isnan(a):
        a = -9e99
    if isnan(b):
        b = -9e99
    if a > b:
        return a
    else:
        return b

# shifts arrays by num possions (without wrapping) (by chrisaycock)
def shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

In [ ]:
# Data read and write (python specific)

dataPath="/asrc/ecr/fabio/NREL_WaterSecurity/ThermalStratification/res-strat-master/data/"
outPath="~/data"

for outdir in ["depth","stratification"]:
    path = os.path.join(outPath,"outputs", outdir) 
    os.makedirs(path, exist_ok = True) 

def LoadData(fileIn):
    # Load the data for the current reservoir

    # Load flow data
    cols=["inflow_re","temp_riv","outflow_re"]
    #flow=np.genfromtxt(os.path.join(dataPath,'inputs','flow',fileIn), dtype=float, delimiter='\t',names=cols)
    flow=pd.read_csv(os.path.join(dataPath,'inputs','flow',fileIn), dtype=float, delimiter='\t',names=cols)
    # and set the index base 1 (we'll use it later in the calculations)
    flow.index=[i for i in range(1,len(flow)+1)]

    # Load climate forcings
    cols=["coszen","lw_abs","s_w","rh","t_air","u_2"]
    #forcing=np.genfromtxt(os.path.join(dataPath,'inputs','forcing',fileIn), dtype=float, delimiter='\t',names=cols)
    forcing=pd.read_csv(os.path.join(dataPath,'inputs','forcing',fileIn), dtype=float, delimiter='\t',names=cols)
    # and set the index base 1 (we'll use it later in the calculations)
    forcing.index=[i for i in range(1,len(forcing)+1)]

    # Load reservoir geometry
    cols=["gm_j","depth","d_ht","M_L","M_W","V_err","Ar_err","C_v","C_a","V_df","A_df","d_n"]
    cols_type={"gm_j":np.int16,
               "depth":np.float32,
               "d_ht":np.float32,
               "M_L":np.float32,
               "M_W":np.float32,
               "V_err":np.float32,
               "Ar_err":np.float32,
               "C_v":np.float32,
               "C_a":np.float32,
               "V_df":np.float32,
               "A_df":np.float32,
               "d_n":np.int16}
    # ['1\t24.2\t39\t16\t3.6\t0\t2\t0.96\t0.97\t-0.08\t-1.1\t4\n']
    #geometry=np.genfromtxt(os.path.join(dataPath,'inputs','geometry',fileIn), dtype=float, delimiter='\t',names=cols)
    geometry=pd.read_csv(os.path.join(dataPath,'inputs','geometry',fileIn), dtype=cols_type, delimiter='\t',names=cols)
    # and set the index base 1 (we'll use it later in the calculations)
    geometry.index=[i for i in range(1,len(geometry)+1)]

    return flow,forcing,geometry

def OutDataframes(TimeSteps,StartLayers):
    nRows=[i for i in range1(1,TimeSteps)]
    colStrat=["T_K_{}".format(i) for i in range(0,StartLayers)]
    colDepth=["TimeStep","NLayers","InFlow","OutFlow","TotDepth","Top0"]+["Top-{}".format(i) for i in range(1,StartLayers)]
    dfStratification=pd.DataFrame(0,index=nRows,columns=colStrat)
    dfDepth=pd.DataFrame(0,index=nRows,columns=colDepth)
    return dfStratification, dfDepth

In [ ]:
# Variables declarations
# Note that all declarations were preserved for reference (some commented out)

# Constants
d_n_max = 30                  # Maximum number of layers
yr_max  = 20                  # Maximum number of years
MAXRES  = 22                  # max reservoirs
NDAYS   = 365                 # days in year
NHRS    = 24                  # hours in day
dtime   = 60                  # number time steps (secs)
s_dtime = dtime*dtime         # number sub-hourly time steps
ddz_min = 1.5                 # maximum layer thickness limit
beta = 0.175e0                # shortwave absorption factor
bias = 1.00e0                 # bias in absorption
zero = 0.0e0                  # initialize arrays and scalars using this

t_frz  = 273.15e0             # freezing temperature (deg=K)
num_fac=1.e6                  #
cfw = 1.e-02                  # ? coefficient of ? water
cfa = 1.e-05                  # ? coefficient of ? air

# Constants not used as array dimensions !!!!
d_n = 30                 # Number of vertical depth discretization with d_n + 1 layer areas yj can this be > d_n_max??? it was set to 50 (not in this version)
d_nn = 1000              # Number of vertical depth discretization to establish depth-area-volume relationship
grav   = 9.8062          # gravity constant (m/s2)
rho_w = 1.e3             # Water density  (kg/m3)
rho_a = 1.177            # Air density  (kg/m3)
c_w   = 4.188e3          # Water specific heat capacity  (J/kg/k)
t_max = 277.             # temperature of maximum water density (K)
st_bl = 5.67e-8          # Stefan-Boltzmann constant ~ W/m^2/K^4
F = 1.0                  # dimensionless factor for wind sheltering by riparian vegetation, Wu et al, 2012

# Arrays
df_eff = dimArray(d_n_max)           # Effective diffusivity (molecular + eddy) [m2/s]
rho_z = dimArray(d_n_max)            # Depth based water density  (kg/m3)
#rho_r                               # Density of inflow water  (kg/m3)
drhodz = dimArray(d_n_max)           # d [rhow] /dz (kg/m**4)
d_zs = dimArray(d_n_max)             # Depth at z from surface (m)
d_zsb = dimArray(d_n_max)            # Depth at z from surface averaged over sub-timestep(m)
d_zf = dimArray(d_n_max)             # Depth at z from surface reversed to top layer(m)
d_z = dimArray(d_n_max)              # Depth at z from bottom (m)
d_s  = 0.60                          # Surface layer depth (m)
m1 = dimArray(d_n_max)               # next 3 arrays used in tridiagonal matrix
m2 = dimArray(d_n_max)
m3 = dimArray(d_n_max)
# AX = dimArray(d_n_max)             # next 5 arrays used in tridiagonal matrix
# BX = dimArray(d_n_max)             # Only FX used in code...
# CX = dimArray(d_n_max)
# DX = dimArray(d_n_max)
FX = dimArray(d_n_max)
a = dimArray(d_n_max)               # "a" left  diagonal of tridiagonal matrix
b = dimArray(d_n_max)               # "b" diagonal column for tridiagonal matrix
c = dimArray(d_n_max)               # "c" right  diagonal tridiagonal matrix
r = dimArray(d_n_max)               # "c" right  diagonal tridiagonal matrix
fac_1 = dimArray(d_n_max)           # Factor for calculation of triadiagonal matrices elements
fac_2 = dimArray(d_n_max)           # Factor for calculation of triadiagonal matrices elements
#real(r8) :: ri                     # Richardson number
a_d = dimArray(d_n_max)             # Area at depth z (km2)
a_dn = dimArray(d_n_max)            # Adjusted area at depth z (km2)
v_z = dimArray(d_n_max)             # Reservoir volume at depth z (m^3)
m_zo = dimArray(d_n_max)            # Reservoir beginning mass at depth z (kg)
#real(r8) :: m_intial               # Reservoir beginning mass  (kg)
#real(r8) :: m_cal                  # Reservoir calculated mass (kg)
#real(r8) :: m_mod                  # Reservoir modeled mass (kg)
#real(r8) :: den                    # Density (kg/m3)
#real(r8) :: m_cal_sub              # Reservoir calculated mass per sub-timestep (kg)
m_znn = dimArray(d_n_max)           # Reservoir # ending mass at depth z averaged for sub-timestep(kg)
m_zno = dimArray(d_n_max)           # Reservoir initial mass at depth z (kg)
m_zn = dimArray(d_n_max)            # Reservoir # ending mass at depth z (kg)
m_zn_old = dimArray(d_n_max)        # Reservoir # ending mass at depth z to be used in halving/merging layers (kg)
m_zn_sub = dimArray(d_n_max)        # Reservoir # ending mass at depth z (kg)
m_in = dimArray(d_n_max)            # Reservoir inflow mass at depth z (kg)
m_in_sub = dimArray(d_n_max)        # Reservoir inflow mass at sub-timestep depth z (kg)
m_ou = dimArray(d_n_max)            # Reservoir outflow mass at depth z (kg)
dm_nt = dimArray(d_n_max)           # net mass added at depth z (kg)
dm_in = dimArray(d_n_max)           # mass added to depth z (kg)
dm_ou = dimArray(d_n_max)           # initial mass removed from depth z (kg)
m_ou_sub = dimArray(d_n_max)        # Reservoir outflow mass at depth z (kg)
#real(r8) :: m_ev,e_cal,e_mod       # initial evaporation mass (kg)
#real(r8) :: m_ev_sub               # Reservoir outflow mass at depth z (kg)
dm_z = dimArray(d_n_max)            # Reservoir mass change at depth z (kg)
dm_z_sub = dimArray(d_n_max)        # Reservoir mass change at depth z (kg)
ds_z = dimArray(d_n_max)            # Reservoir volume change at depth z (kg)
ds_z_sub = dimArray(d_n_max)        # Reservoir volume change at depth z (kg)
d_v = dimArray(d_n_max)             # Reservoir volume change at layer (m^3)
dv_in = dimArray(d_n_max)           # volume increment at layer due to inflow(m^3)
dt_in = dimArray(d_n_max)           # temperature increment at layer due to inflow(k)
dv_ou = dimArray(d_n_max)           # volume decrease at layer due to inflow(m3)
dt_ou = dimArray(d_n_max)           # temperature decrease at layer due to inflow(k)
#real(r8) :: v_t                    # Total storage (m^3)
#real(r8) :: v_evap                 # Evaporated volume (m^3)
#real(r8) :: d_evap                 # Evaporated depth (m)
#real(r8) :: delta_z                # depth change to calculate corresponding area/volume(m)
#real(r8) :: top_d                  # top layer depth
v_zt = dimArray(d_n_max)            # Total reservoir volume at depth z from surface(m3)
#real(r8) :: v_mix                  # Total volume of mixed layer(m3)
#integer :: nmix                    # Mixed layer(-)
#integer :: Hday,Jday               # Hour of the day, Julian day (-)
#real(r8) :: in_tk                  # Inflow zone thickness (m)
#real(r8)   :: in_f,in_t,ou_f       # Inflow(m^3/s), inflow temp(k) and outflow(m^3/s)
#real(r8) :: s_tin                  # Initial total storage (m^3)
#real(r8) :: s_t                    # Total storage at timestep t (m^3)
#real(r8) :: ds_t                   # change in storage (m^3)
#real(r8) :: t_s                    # surface temperature (k) -hourly
#real(r8) :: t_sdy                  # surface temperature (k) - for daily conversion
t_z = dimArray(d_n_max)             # lake layer temperature
t_zr = dimArray(d_n_max)            # lake layer temperature rounded to three decimal places
t_zf = dimArray(d_n_max)            # lake layer temperature to be written to a file adjusted so that surface temp will be at layer 50
t_zs = dimArray(d_n_max)            # lake layer temperature averaged over sub-timestep
t_z_old = dimArray(d_n_max)         # previous time lake layer temperature
t_zsub = dimArray(d_n_max)          # sub-timestep lake layer temperature (k)
dd_in = dimArray(d_nn)              # Initial layer depth(m)
dd_z = dimArray(d_n_max)            # Layer depth(m)
dd_inc = dimArray(d_n_max)          # Depth increment (m)
#real(r8) :: dd_max                 # Maximum depth change (m)
#real(r8) :: ddz_min,ddz_max        # Minimum and maximum layer thickness limit
#real(r8) :: bv_f                   # brunt-vaisala frequency [s**2)
#real(r8) :: eta                    # light extinction coefficient
#real(r8) :: sh_net                 # net short wave radiation
#real(r8) :: bias                   # bias correction for solar radiation
#real(r8) :: sh_mix                 # net short wave radiation in mixed layer
#real(r8) :: beta                   # shortwave absorbtion factor
#real(r8) :: lw_abs                 # atmospheric longwave absorbtion (W/m^2)
#real(r8) :: lw_abr                 # atmospheric longwave absorbtion for sub-timestep (W/m^2)
#real(r8) :: phi_o                  # net surface radiation (W/m^2)
phi_z = dimArray(d_n_max)           # radiation absorbed by layer (W/m^2)
phi_zf = dimArray(d_n_max)          # radiation absorbed by layer saved to file so that surface is layer 50
phi_x = dimArray(d_n_max)           # radiation absorbed by mixed layer (W/m^2)
phi_zsub = dimArray(d_n_max)        # sub-timestep radiation absorbed by layer (W/m^2)
#real(r8) :: lw_ems                 # longwave emission
#real(r8) :: lt_heat                # latent heat
le= 2.501e6                         # latent heat of vaporaization (kcal/kg)
#real(r8) :: sn_heat                # sensible heat
#real(r8) :: emt                    # average emittance
#real(r8) :: alb_s                  # surface albedo shortwave radiation
#real(r8) :: k_m                    # molecular diffusivity
enr_ol = dimArray(d_n_max)          # Starting layer energy
enr_nw = dimArray(d_n_max)          # # ending energy change
enr_in = dimArray(d_n_max)          # Layer Energy from inflow
enr_ou = dimArray(d_n_max)          # Layer energy from outflow
enr_0 = dimArray(d_n_max)           # Initial inner energy
enr_1 = dimArray(d_n_max)           # Inner energy after advection
enr_2 = dimArray(d_n_max)           # Inner energy after stratification
enr_2c = dimArray(d_n_max)          # Inner energy after stratification and convective mixing
enr_0r = dimArray(d_n_max)          # Next 5 arrays are: Area rated energy (w/m^2)
enr_inr = dimArray(d_n_max)
enr_our = dimArray(d_n_max)
enr_1r = dimArray(d_n_max)
enr_2r = dimArray(d_n_max)
enr_0sub = dimArray(d_n_max)        # Next 5 arrays are: sub-timestep energy (w)
enr_insub = dimArray(d_n_max)
enr_ousub = dimArray(d_n_max)
enr_1sub = dimArray(d_n_max)
enr_2sub = dimArray(d_n_max)
#real(r8) :: enr_err                # Energy balance error (W/m^2)
#real(r8) :: enr_phi                # Energy from surface net flux before conv mixing
#real(r8) :: enr_phic               # Energy from surface net flux after conv mixing
#real(r8) :: enr_err2ca             # Energy error (w/m2)
#real(r8) :: sn_heatc               # Energy from sensible heat flux after conv mixing
#real(r8) :: s_err                  # Water balance error (m^3)
#real(r8) :: enr_err1,enr_err2,enr_err2c # Energy error (w) before stratification, after triadiagonal solution, and after convective mixing
#real(r8) :: k_eff,k_ew,k_ec
k_ad = dimArray(d_n_max)            # Effective,wind,convection,advective kinetic energy (kg.m^2/s^2)
#real(r8) :: c_d                    # Drag coefficient
#real(r8) :: cfa,cfw                # ,cft
Fr = dimArray(d_n_max)              # Froude number squared and inverted for diffusion coeff. calculation
dis_ad = dimArray(d_n_max)
th_en = dimArray(d_n_max)           #Layer thermal energy (j/s)
#real(r8) :: dis_w
#real(r8) :: l_vel
#real(r8) :: s_vel
#real(r8) :: tau
#real(r8) :: th_ex,enr_bc,enr_ac
cntr = dimArray(d_n_max)            # Next 3 arrays, counters to average sub-timestep temperature
cntr1 = dimArray(d_n_max)
cntr2 = dimArray(d_n_max)
q_adv = dimArray(d_n_max)           # Layer flow rate (m^3/s)
z_str = dimArray(d_n_max)           # Z* for layer solar radiation calculation (m)
dd_z_old = dimArray(d_n_max)
d_v_old = dimArray(d_n_max)
dv_in_old = dimArray(d_n_max)
dv_ou_old = dimArray(d_n_max)
#real(r8) :: m_diff
m_cor = dimArray(d_n_max)
#d_zi = dimArray(d_nn+1)               # Next 3 arrays: Initial depth, area, and volume for reservoir geometry
#a_di = dimArray(d_nn+1)
#v_zti = dimArray(d_nn+1)
#integer  :: d_n_n                  # Adjusted layer number
d_zn = dimArray(d_n_max)            # Next 3 arrays: Adjusted layer depth, volume, and temperature
d_vn = dimArray(d_n_max)
v_ztn = dimArray(d_n_max)
t_zn = dimArray(d_n_max)
#real(r8) :: denmix,denst,tmix,sumvol,tsum,vlmxlw,vlmxtp
#real(r8) :: msmxlw,msmxtp,summas,enmxlw,enmxtp,sumenr
#integer :: mixlow,mixtop
#real(r8) :: sh_neta,lw_absa,lt_heata,lw_emsa,sn_heata,phi_oa,st_sub
#real(r8) :: ta,tb,tab,dd_za,dd_zb,d_va,dv_oua,delta_a,delta_v,e_a,e_b,e_ab
dv_nt = dimArray(d_n_max)
#real(r8) :: dv_inb,dv_ina,dv_oub,dv_ouab,dv_inab,dd_zab,d_vab,d_vb,m_ab,m_a,m_b,num_fac,num_fac1
#real(r8) :: rho_ztm
t_zt = dimArray(d_n_max)
rho_zt = dimArray(d_n_max)
#real(r8) :: zmix,f_ri,by_e,cmzmix,drhomx,t_zmix
cmz = dimArray(d_n_max)
#real(r8) :: s_err_sub,enr_err_sub  #Sub-timestep storage and energy error
#real(r8) :: V_err,Ar_err,V_df,A_df #Volume error (%), area error(\), volume difference(mcm), and area difference (km2) (+ve difference means underestimation)
#real(r8) :: A_cf,V_cf              #Area and volume correcting factor for error from geometry estimation
#real(r8) :: lat,lon                #Latitude and longitude  of reservoir grid
#real(r8) :: DECL,DELTS,HSR,sina, SUNSET,SUNUP,T1,T2
#integer :: flag, countmax,countmin

In [ ]:
# Functions related to the model
def solve(a, b, c, r, u, n):  # u, is the return variable #MNLAKE
    #    implicit none
    #	 a - left coefficient , b - middle coefficient, c - right coefficient
    #	 r - right hand side (known), u - the answer (unknown), n - number of equations
    # integer,parameter :: r8 = selected_#real_kind( 16) # 8 byte real
    # integer,intent(in) :: n
    # real(r8),dimension(n),intent(in) :: a,b,c,r
    # real(r8),dimension(n),intent(out) :: u
    # real(r8),dimension(n) :: bp,rp
    # real(r8) :: m,tt
    # integer i
    #     initialize c-prime and d-prime
    bp = dimArray(n)
    rp = dimArray(n)
    #u = dimArray(n)
    bp[1] = b[1]
    rp[1] = r[1]
    for i in range1(2, n):
        tt = a[i] / b[i - 1]
        bp[i] = b[i] - c[i - 1] * tt
        rp[i] = r[i] - rp[i - 1] * tt
    bp[n] = b[n]
    # rp[n]=r[n]
    # initialize u
    u[n] = rp[n] / bp[n]

    # Back substitution
    for i in range(n - 1, 1, -1):
        u[i] = (rp[i] - c[i] * u[i + 1]) / bp[i]
    return u

def LayerChar_d(TestValue,dfGeom,d_nn): # Testing by Depth

    d_zi_d_nn, a_di_d_nn, v_zti_d_nn = dfGeom.loc[dfGeom.index == d_nn, ['d_zi', 'a_di', 'v_zti']].values[0]
    d_zi_d_nn_p1, a_di_d_nn_p1, v_zti_d_nn_p1 = dfGeom.loc[dfGeom.index == d_nn + 1, ['d_zi', 'a_di', 'v_zti']].values[0]

    layer = dfGeom.loc[(dfGeom['d_zi'].shift(1) < TestValue) & (dfGeom['d_zi'] >= TestValue)].index
    if len(layer) > 0:
        layer = layer[0]
        d_zij, a_dij, v_ztij = dfGeom.loc[dfGeom.index == layer, ['d_zi', 'a_di', 'v_zti']].values[0]
        d_zij_1, a_dij_1, v_ztij_1 = dfGeom.loc[dfGeom.index == (layer - 1), ['d_zi', 'a_di', 'v_zti']].values[0]
        if (d_zij - d_zij_1) == 0.:
            print("Division by 0 at {}".format(layer))
        delta_z = (TestValue - d_zij_1) / (d_zij - d_zij_1)
        a_d_local = delta_z * (a_dij - a_dij_1) + a_dij_1
        v_zt_local = delta_z * (v_ztij - v_ztij_1) + v_ztij_1
    else:
        if (TestValue > d_zi_d_nn_p1): # and (i <= d_n + 1):  # commented out the 2nd condition because it never happens
            delta_z = (TestValue - d_zi_d_nn) / (d_zi_d_nn_p1 - d_zi_d_nn)
            a_d_local = delta_z * (a_di_d_nn_p1 - a_di_d_nn) + a_di_d_nn
            v_zt_local = delta_z * (v_zti_d_nn_p1 - v_zti_d_nn) + v_zti_d_nn
        else:
            print("Nothing to do in function LayerChar_d")
    return a_d_local,v_zt_local

def LayerChar_v(TestValue,dfGeom,d_nn): # Testing by Volume

    d_zi_d_nn, a_di_d_nn, v_zti_d_nn = dfGeom.loc[dfGeom.index == d_nn, ['d_zi', 'a_di', 'v_zti']].values[0]
    d_zi_d_nn_p1, a_di_d_nn_p1, v_zti_d_nn_p1 = dfGeom.loc[dfGeom.index == d_nn + 1, ['d_zi', 'a_di', 'v_zti']].values[0]

    layer = dfGeom.loc[(dfGeom['v_zti'].shift(1) < TestValue) & (dfGeom['v_zti'] >= TestValue)].index
    if len(layer) > 0:
        layer = layer[0]
        d_zij, a_dij, v_ztij = dfGeom.loc[dfGeom.index == layer, ['d_zi', 'a_di', 'v_zti']].values[0]
        d_zij_1, a_dij_1, v_ztij_1 = dfGeom.loc[dfGeom.index == (layer - 1), ['d_zi', 'a_di', 'v_zti']].values[0]
        if (v_ztij - v_ztij_1) == 0.:
            print("Division by 0 at {}".format(layer))
        delta_z = (d_zij - d_zij_1) * (TestValue - v_ztij_1) / (v_ztij - v_ztij_1)
        delta_a = (a_dij - a_dij_1) * (TestValue - v_ztij_1) / (v_ztij - v_ztij_1)
        d_z_local = d_zij_1 + delta_z
        a_d_local = a_dij_1 + delta_a
    else:
        if (TestValue > v_zti_d_nn_p1):
            delta_z = (d_zi_d_nn_p1 - d_zi_d_nn) * (TestValue - v_zti_d_nn) / (v_zti_d_nn_p1 - v_zti_d_nn)
            delta_a = (a_di_d_nn_p1 - a_di_d_nn) * (TestValue - v_zti_d_nn) / (v_zti_d_nn_p1 - v_zti_d_nn)
            d_z_local = d_zi_d_nn + delta_z
            a_d_local = a_di_d_nn + delta_a
        else:
            print("Nothing to do in function LayerChar_v")
    return a_d_local,d_z_local


def rgeom(M_W, M_L, gm_j, d_res, dd_in, d_nn, C_a, C_v,A_cf,V_cf):  # these are the return variables: d_zi,a_di,v_zti):
    
    ####################################################################
    # Edited to remove long loops, replaced with numpy array operation #
    # Original code commented and left in reference                    #
    ####################################################################
    
    # Calculate reservoir layer average area (km2)
    # implicit none
    # integer,parameter :: r8 = selected_#real_kind( 16) # 8 byte real
    # real(r8), intent(in)  :: M_W,M_L,gm_j,d_res,dd_in(1000),C_a,C_v#,V_cf,A_cf
    # integer, intent(in)  :: d_nn        #
    # real(r8),dimension(1001), intent(out) :: d_zi,a_di,v_zti
    # real(r8) ::dd_zz(1000),a_dd(1001),a_zi(1001), ar_f = 1.0e6 ,C_aa(1001)              # Factor to convert area to m^2
    # real(r8) :: pi      = 3.14159265358979323846    # pi
    # real(r8) :: dv,da,dz
    # integer :: j,k        # indices
    d_zi = dimArray(d_nn+1)
    a_di = dimArray(d_nn+1)
    v_zti = dimArray(d_nn+1)
    #dd_zz = dimArray(d_nn+1)
    a_dd = dimArray(d_nn+1)
    a_zi = dimArray(d_nn+1)
    C_aa = dimArray(d_nn)
    jj=np.array([float(i) for i in range(0,d_nn+1)])
    ar_f = 1.0e6
    pi = 3.14159265358979323846
    #for j in range1(1, d_nn):
    #    dd_zz[j] = dd_in[j]
    #dd_zz[1:d_nn+1] = dd_in[1:d_nn+1]
    dd_zz=dd_in

    #for j in range1(d_nn + 1, 1, -1):
    #    C_aa[j] = 1. * C_a  #
    C_aa[1:]=1. * C_a
    
    # Calculate depth area
    # ******************************************* Curved Lake Bottom ****************************************************************
    #for j in range1(1, d_nn):
    if (gm_j == 1): # .0):
        #a_dd[j] = C_aa[j] * M_L * M_W * (1 - ((dd_zz[j] * (j - 1)) / d_res) ** 2.)
        a_dd[1: -1]=(C_aa * M_L * M_W * (1 - ((dd_zz * shift(jj, 1)) / d_res) ** 2.))[1:]
    elif (gm_j == 3): # .0):
        #a_dd[j] = C_aa[j] * M_L * M_W * (1 - ((dd_zz[j] * (j - 1)) / d_res) ** 2.) * (
        #        (d_res - (dd_zz[j] * (j - 1))) / d_res) ** 0.5
        a_dd[1: -1] = (C_aa * M_L * M_W * (1 - ((dd_zz * shift(jj, 1)) / d_res) ** 2.) * (
                 (d_res - (dd_zz * shift(jj, 1))) / d_res) ** 0.5)[1:]
    elif (gm_j == 5): # .0):
        #a_dd[j] = C_aa[j] * pi * 0.25 * M_L * M_W * (1 - ((dd_zz[j] * (j - 1)) / d_res) ** 2.) * (
        #        (d_res - (dd_zz[j] * (j - 1))) / d_res) ** 0.5
        a_dd[1: -1] = (C_aa * pi * 0.25 * M_L * M_W * (1 - ((dd_zz * shift(jj, 1)) / d_res) ** 2.) * (
                   (d_res - (dd_zz * shift(jj, 1))) / d_res) ** 0.5)[1:]
    elif (gm_j == 2): # .0):
        #a_dd[j] = C_aa[j] * M_L * M_W * (1 - ((dd_zz[j] * (j - 1)) / d_res) ** 2.) * (
        #            1 - ((dd_zz[j] * (j - 1)) / d_res))
        a_dd[1: -1] = (C_aa * M_L * M_W * (1 - ((dd_zz * shift(jj, 1)) / d_res) ** 2.) * (
                    1 - ((d_res - (dd_zz * shift(jj, 1))) / d_res)))[1:]
    elif (gm_j == 4): # .0):
        #a_dd[j] = C_aa[j] * (2. / 3.) * M_L * M_W * (1 - ((dd_zz[j] * (j - 1)) / d_res) ** 2.) * (
        #        1 - ((dd_zz[j] * (j - 1))) / d_res)
        a_dd[1: -1] = (C_aa * (2. / 3.) * M_L * M_W * (1 - ((dd_zz * shift(jj, 1)) / d_res) ** 2.) * (
                1 - ((d_res - (dd_zz * shift(jj, 1))) / d_res)))[1:]
    else:
        raise Exception('Unknown reservoir shape {}'.format(gm_j))
        ## end if
    a_dd[d_nn + 1] = 0.1  # Bottom area given non-zero value

    # ***********************************************************************************************************

    # Reverse indexing so that bottom is 1 and top is d_n+1 and convert to m2
    #for j in range1(1, d_nn + 1):
    #    k = d_nn + 2 - j
    #    a_di[k] = max(a_dd[j], 1.) * ar_f
    a_di[1:]=np.flip(a_dd)[:-1]
    # Replaced max function with numpy filter
    a_di[np.where((a_di > 0) & (a_di < 1)) ] = 1
    a_di = a_di * ar_f

    a_di[1] = 0.1  # Bottom Area

    if (a_di[d_nn + 1] < a_di[d_nn]):
        a_di[d_nn + 1] = a_di[d_nn]

    # 	Calculate layer depth from bottom,area,and volume
    d_zi[1] = 0.
    #for j in range1(2, d_nn + 1):
    #    d_zi[j] = d_zi[j - 1] + dd_in[j - 1]
    d_zi[2:]=np.cumsum(d_zi[1:-1] + dd_in[1:])

    # 	Calculate layer average area,and total volume from bottom
    v_zti[1] = 0.1
    #for j in range1(2, d_nn + 1):
    #    a_zi[j - 1] = max(0.5 * (a_di[j] + a_di[j - 1]), 1.)  # Area converted to m^2
    #    v_zti[j] = (v_zti[j - 1] + C_v * a_zi[j - 1] * dd_in[j - 1])
    #    # v_zti(j) = v_zti(j-1) + C_v*a_zi(j-1)*dd_in(j-1)+V_df/d_nn
    a_zi[1:-1] = 0.5 * (a_di[1:] + shift(a_di,1)[1:])[1:]
    v_zti[2:]=np.cumsum(v_zti[1:-1] + C_v * a_zi[1:-1] * dd_in[1:])


    a_zi[d_nn + 1] = a_di[d_nn + 1]

    # do j = 1,d_nn+1
    # print*,d_zi(j),a_di(j),v_zti(j)
    #
    # stop
    # # end def rgeom

    # Moved out of the main code
    a_di = A_cf * a_di  # Area corrected for error
    v_zti = V_cf * v_zti  # Volume corrected for error

    cols=['d_zi','a_di','v_zti','sort_top_up','sort_top_down']
    index=[i for i in range1(1,d_nn+1)]
    index_r=[i for i in range1(d_nn+1,1,-1)]
    dfGeom=pd.DataFrame(index=index,columns=cols)
    dfGeom['sort_top_down']=index
    dfGeom['sort_top_up']=index_r
    dfGeom['d_zi'] = d_zi[1:]
    dfGeom['a_di'] = a_di[1:]
    dfGeom['v_zti'] = v_zti[1:]
    # dfGeom['a_zi'] = a_zi[1:]
    # dfGeom['dd_zz'] = dd_zz
    # dfGeom['dd_zz'] = dfGeom['dd_zz'].shift(-1)

    return dfGeom # d_zi, a_di, v_zti

def flowdist(d_n, in_f, in_t, ou_f, dtime, d_v, v_zt, c_w):  # Return variables: dv_in,dv_ou,dm_in,th_en
    # *******************************************************************************************************
    # 	Calculation inflow/outflow contribution adopted from CE-QUAL-R1 model
    # *******************************************************************************************************
    # implicit none
    # integer,parameter :: r8 = selected_#real_kind( 16) # 8 byte real
    # integer,parameter :: d_n_max = 30
    d_n_max = 30
    # integer, intent(in)  :: d_n,dtime        #
    # real(r8), intent(in)  :: in_f,in_t,ou_f,c_w,d_v(d_n_max),v_zt(d_n_max)
    # real(r8),dimension(d_n_max), intent(out) :: dv_in,dv_ou,dm_in,th_en    # layer inflow/outflow (m3/s)
    # real(r8) :: rho_r,in_v,m_in    #
    # integer :: j,jmax,jmin        			# indices
    # #integer, intent(out)  :: jminn,jmaxx
    dv_in = dimArray(d_n_max,0)
    dv_ou = dimArray(d_n_max,0)
    dm_in = dimArray(d_n_max,0)
    th_en = dimArray(d_n_max,0)
    rho_r = 1000. * (1.0 - 1.9549e-05 * (abs(in_t - 277.)) ** 1.68)
    in_v = in_f * dtime
    m_in = in_v * rho_r
    #	Initialize - We already initialize the arrays to 0
    #  for j in range(0, d_n_max):
    #    dv_in[j]=0.
    #    dv_ou[j]=0.
    #    dm_in[j]=0.
    #    th_en[j]=0.

    #   Layer inflow and energy contribution
    jmin = 1
    if (d_n > 3):
        jmax = d_n - 3
    else:
        jmax = d_n - 1

    if (d_n == 1):  # Single layer
        dv_in[:] = 0.
        dm_in[:] = 0.
        th_en[:] = 0.
        dv_ou[:] = 0.
        #for j in range1(1,d_n_max):
        #    dv_in[j] = 0.
        #    dm_in[j] = 0.
        #    th_en[j] = 0.
        #    dv_ou[j] = 0.
        dv_in[d_n] = in_f
        dm_in[d_n] = m_in
        th_en[d_n] = dv_in[d_n] * rho_r * in_t * c_w
        dv_ou[d_n] = ou_f
    else:
        denom = v_zt[jmax + 1] - v_zt[jmin]
        dv_in[jmin:jmax+1] = in_f * (d_v[jmin:jmax+1] / denom)
        dm_in[jmin:jmax+1] = m_in * (d_v[jmin:jmax+1] / denom)
        th_en[jmin:jmax+1] = dv_in[j] * rho_r * in_t * c_w
        dv_ou[jmin:jmax+1] = ou_f * (d_v[jmin:jmax+1] / denom)        
        #for j in range1(jmin, jmax):
        #    dv_in[j] = in_f * (d_v[j] / (v_zt[jmax + 1] - v_zt[jmin]))
        #    dm_in[j] = m_in * (d_v[j] / (v_zt[jmax + 1] - v_zt[jmin]))
        #    th_en[j] = dv_in[j] * rho_r * in_t * c_w
        #    dv_ou[j] = ou_f * (d_v[j] / (v_zt[jmax + 1] - v_zt[jmin]))

    ## end if

    # end subroutine flowdist
    return dv_in, dv_ou, dm_in, th_en # jminn,jmaxx

def den(t_z):  # result(rho)
    # calculate density from temperature
    # implicit none
    # integer,parameter :: r8 = selected_#real_kind( 16) # 8 byte real
    # real(r8), intent(in) :: t_z# Temperature (k)
    # real(r8) :: rho# # density (kg/m3)

    # modified from Subin et al, 2011 with lake ice fraction = 0
    rho = 1000. * (1.0 - 1.9549e-05 * (abs(t_z - 277.)) ** 1.68)

    return rho

In [ ]:
# Read reservoir data into a dataframe (python specific)
# Lat and Lon are not used in the code, but the original program
# kept them separate, so we do the same (no time cost)
reservoirFile=os.path.join(dataPath,"inputs","reservoirs.txt")
reservoirs=pd.read_csv(reservoirFile,header=None,names=['ReservoirFile'])
reservoirs['lon']=reservoirs['ReservoirFile'].apply(lambda x: str(x)[:-4].split("_")[1])
reservoirs['lat']=reservoirs['ReservoirFile'].apply(lambda x: str(x)[:-4].split("_")[0])


# Main loop of the model.

Loops through:

-  Reservoirs
    -  Time steps (as defined by the forcings files). In the case of the sample data provided with the code these are 87600 (10 years at hourly time steps, not counting leap years)
        -  Sub-time steps. Set to 60, which intuitively should indicate minutes. However the explanation for the sub-time steps is for "numerical stability" __THIS IS ONE OF THE FIRST THINGS TO ASSESS... CAN WE REDUCE THIS SUB-TIME STEPS?__

The note in the code (line 127 below) seems to indicate that when the forcing data is provided at the daily time steps, the sub-time steps should be set to something different, but does not elaborate on that...

### Note 
* you can visualize the line numbers if you enable the debug mode (little bug icon at the top right of this pane...). In debug mode you can also set breakpoints to step through the code and check the values of the varliabes as you do... __However, debug mode will make the code run much slower...__ 
- The loop is curently set to run for only the first reservoir and for the first 72 hours... __This can be changed at the beginning of the loops al line 4 to 10 (reservoirs) and 131 to 135 (time steps) in the code cell below...__

In [ ]:
# this is the main loop!!
# Now we loop thorugh the reservoirs

#for res_index,reservoir in reservoirs.iterrows():

# Commenting the line above and uncommenting the line below 
# allow to do runs for specific reservoirs (using the record number/s (0 based)
# of the resevoir/s to model)

for res_index, reservoir in reservoirs.iloc[0].to_frame().transpose().iterrows():

    break4099 = False  # Used to structure the most external goto...

    energy = 'depth_'
    fileIn = reservoir['ReservoirFile']

    # Read the data into Pandas Data Frames
    flow, forcing, geometry = LoadData(fileIn)

    # Extract the current reservoir's geometry
    gm_j, depth, d_ht, M_L, M_W, V_err, Ar_err, C_v, C_a, V_df, A_df, d_n = geometry.iloc[0].values

    # Do some adjustments (gm_j - shape index and d_n number of layers
    # need to be integers)
    gm_j = int(gm_j)
    d_n = int(d_n)
    d_n_out=d_n

    # Initialize output Pandas dataframes (
    dfStrat, dfDepth = OutDataframes(len(forcing), d_n_out)

    # Allows for setting the dam height to 0 (maybe works also for 
    # non-artificial lakes??) and intializes dam height to depth...
    if d_ht <= 0.0:
        d_ht = depth
    # Reservoir "operational" depth set to 95% of the dam height
    d_res = 0.95 * d_ht

    # Calculate layer depth
    if (M_W <= 0.0):
        M_W = 1.
    if (M_L <= 0.0):
        M_L = 1.

    # Uniform subsurface layer depth for initialization and limit 
    # maximum layer thickness TODO not sure it needs to be 1000
    #for j in range1(1, d_nn):
    #    dd_in[j] = d_res / d_nn  # bottom layers evenly descritized
    dd_in[1:d_nn+1] = d_res / d_nn  # bottom layers evenly descritized

    A_cf=V_cf=None
    # Area and volume correcting factors for relative error as compared to GRanD
    if (A_df >= 0):
        A_cf = 1. + (abs(Ar_err) / 100.)
    elif (A_df < 0):
        A_cf = 1. - (abs(Ar_err) / 100.)

    if (V_df >= 0):
        V_cf = 1. + (abs(V_err) / 100.)
    elif (V_df < 0):
        V_cf = 1. - (abs(V_err) / 100.)

    # Calculate reservoir geometry to establish depth-area-volume relationship
    dfGeom = rgeom(M_W, M_L, gm_j, d_res, dd_in, d_nn, C_a, C_v, A_cf, V_cf)
    #d_zi, a_di, v_zti = rgeom(M_W, M_L, gm_j, d_res, dd_in, d_nn, C_a, C_v)

    # ***********************************
    # for j in range1(1, d_nn + 1): # TODO remove loop and simply multiply the array by the constant
    # The following lines have been moved into the rgeom function
    #a_di = A_cf * a_di  # Area corrected for error
    #v_zti = V_cf * v_zti  # Volume corrected for error
    
    # We reset because the same variables are later used in the code...
    # although we need to check if they are used as multipliers 
    # (which we could remove...)
    A_cf = 1.
    V_cf = 1.
    # ***********************************

    # The logic in the section below is rather convoluted and could
    # use some simplifying, but it's only ran once per reservoir, so
    # not too urgent...
    
    dd_z[d_n] = d_s  # top layer depth kept constant
    for j in range1(d_n, 1, -1): # TODO Logic can be changed to 1) set the top layer, 2) set all the other layers
        if (j == d_n) and (d_n == 1):
            dd_z[j] = d_res
        elif (((d_n > 1) and (j < d_n)) and (d_res - dd_z[d_n]) > 0.):
            dd_z[j] = (d_res - dd_z[d_n]) / (d_n - 1)  # bottom layers evenly descritized

    if ((d_n > 1) and (dd_z[d_n - 1] < d_s)):  # layer thickness too small
        d_n = int(d_res / d_s) + 1
        for j in range1(1, d_n):
            dd_z[j] = 0.

        # Reinitialize layer thickness
        for j in range1(d_n, 1, -1):
            if (j == d_n):
                dd_z[j] = d_s  # top layer depth = 0.6m
            else:
                dd_z[j] = (d_res - dd_z[d_n]) / (d_n - 1)  # bottom layers evenly descritized

    # Calculate maximum and minimum layer thickness
    if (d_n >= 15):
        ddz_min = 1.5
        if (2.5 * dd_z[d_n - 1] > ddz_min):
            ddz_max = 5. * dd_z[d_n - 1]
        else:
            ddz_max = 2. * ddz_min
    elif ((d_n > 1) and (d_n < 15)):
        ddz_min = 1.5
        if (2.5 * dd_z[d_n - 1] > ddz_min):
            ddz_max = 2.5 * dd_z[d_n - 1]
        else:
            ddz_max = 2. * ddz_min
    elif (d_n == 1):
        ddz_min = 1.5
        ddz_max = 2. * d_res
    m_cal = 0.
    Jday = 0
    flag = 0
    countmax = 0
    countmin = 0

    # Calculate sub-time step for numerical stability
    dtime = 60
    s_dtime = int(3600 / dtime)  # Sub-hourly timestep. Change 3600 if the forcing data is not hourly
    # ************************Start calculation for each timestep	*************************************

    # do ti = 1,fr_ln
    #for ti, forc in forcing.iterrows():
    # The line above runs through all the time steps in the forcings file
    # The one below runs for only 3 days (72 hours) to limit runtime...
    # comment/uncomment as needed
    for ti, forc in forcing.iloc[:72].iterrows():
        # Initialize
        sh_neta = 0.
        lw_absa = 0.
        lt_heata = 0.
        lw_emsa = 0.
        sn_heata = 0.
        phi_oa = 0.
        s_err_sub = 0.
        enr_err_sub = 0.
        m_ev_sub = 0.
        dm_z_sub = 0.
        ds_z_sub = 0.
        m_cal_sub = 0.
        d_res_sub = 0.
        st_sub = 0.
        # for j in range1(1, d_n_max):
        # Convert the loop to reassignment (faster)
        t_zsub[:] = 0.
        phi_zsub[:] = 0.
        m_zn_sub[:] = 0.  # Reservoir ending mass  at depth z (kg)
        m_in[:] = 0.      # Reservoir outflow mass at depth z (kg)
        m_in_sub[:] = 0.  # Reservoir outflow mass at depth z (kg)
        m_ou[:] = 0.      # Reservoir outflow mass at depth z (kg)
        m_ou_sub[:] = 0.  # Reservoir outflow mass at depth z (kg)
        cntr[:] = 0
        cntr1[:] = 0
        cntr2[:] = 0
        d_zsb[:] = 0.

        # coszen=forc(ti,1)
        # lw_abs=forc(ti,2)
        # s_w=forc(ti,3)
        # rh=forc(ti,4)
        # t_air=forc(ti,5)
        # u_2=forc(ti,6)
        # Load the forcing data from the Pandas dataframe
        coszen, lw_abs, s_w, rh, t_air, u_2 = forc.values

        # Index to match daily inflow/outflow value to hourly forcing
        if ((ti % 24) == 0):  # mod(ti,24)
            io = int(ti / 24)
        else:
            io = int(ti / 24) + 1

        # Calculate julian day
        Jday = io % 365  # mod(io,365)
        Hday = ti % 24  # mod(ti,24)
        if ((io % 365) == 0):  # mod(io,365)
            Jday = 365
        if ((ti % 24) == 0):  # mod(ti,24)
            Hday = 24
        
        # Load the flow info from the dataframe
        # in_f=in_re[io,1]
        # in_t=in_re[io,2]
        # ou_f=in_re[io,3]
        in_f, in_t, ou_f = flow.loc[flow.index == io].values[0]
        
        ########################################################
        # The following lines were moved from the inner loop   #
        # because they did not depend on it..                  #
        # Also reorganized and merged some of the if statments #
        # that originally were kept separate without reason    #
        ########################################################

        if ti == 1: # and (ww == 1):  
            #for j in range1(1, d_n_max):  # TODO Reinitializing the array with np.zeros is likely much faster
            m_zo.fill(0) # [j] = 0.  # Reservoir beginning mass at depth z (kg)
            m_zn.fill(0) # [j] = 0.  # Reservoir ending mass at depth z (kg)

        #if (ti == 1) and (ww == 1):  # TODO see above regarding if statement (merge??)
            # Calculate layer depth (minimum at bottom)
            #d_z[1] = 0.
            d_z.fill(0)
            #for j in range1(2, d_n + 1): #d_n_max):  # TODO limit loop to d_n+1, remove if, set rest of array to 0
            #    # with np.zeros
            #    #if (j <= d_n + 1):
            #    d_z[j] = d_z[j - 1] + dd_z[j - 1]
            #    #else:
            #    #    d_z[j] = 0.
            d_z[2: d_n + 2] = np.cumsum(d_z[1:d_n+1] + dd_z[1:d_n+1])
            # Assign layer area and and volume based on depth-area-volume relationship
            #for j in range1(1, d_n_max):  # TODO use array initialization with np.zeros
            a_d.fill(0) #[j] = 0.
            v_zt.fill(0) #[j] = 0.
            d_v.fill(0) #[j] = 0.
            dd_z.fill(0) #[j] = 0.

            #for i in range1(2, d_n + 1):
            #    a_d[i],v_zt[i]=LayerChar_d(d_z[i], dfGeom, d_nn)
            #    #a_d,v_zt=layers(dfGeom, d_z, d_n, d_nn)

            a_d[1] = 0.1
            v_zt[1] = 0.1
            ##layer_last=dfGeom.iloc[-1].index
            ##d_zi_d_nn=dfGeom.iloc[-2].d_zi
            #d_zi_d_nn, a_di_d_nn,v_zti_d_nn=dfGeom.loc[dfGeom.index == d_nn,['d_zi', 'a_di', 'v_zti']].values[0]
            #d_zi_d_nn_p1, a_di_d_nn_p1,v_zti_d_nn_p1=dfGeom.loc[dfGeom.index == d_nn + 1,['d_zi', 'a_di', 'v_zti']].values[0]
            for i in range1(2, d_n + 1):  # TODO check use of elif condition (subject to rounding issues)
                a_d[i],v_zt[i]=LayerChar_d(d_z[i], dfGeom, d_nn)
            #    layer = dfGeom.loc[(dfGeom['d_zi'].shift(1) < d_z[i]) & (dfGeom['d_zi'] >= d_z[i])].index
            #    if len(layer) > 0:
            #        layer = layer[0]
            #        #print("d_z[i] = {}".format(d_z[i]))
            #        d_zij, a_dij, v_ztij = dfGeom.loc[dfGeom.index == layer, ['d_zi', 'a_di', 'v_zti']].values[0]
            #        d_zij_1, a_dij_1, v_ztij_1 = \
            #        dfGeom.loc[dfGeom.index == (layer - 1), ['d_zi', 'a_di', 'v_zti']].values[0]
            #        #print("id, d_zij, d_zij_1 = {}, {}, {}".format(layer, d_zij, d_zij_1))
            #        #print("a_dij, a_dij_1 = {}, {}".format(a_dij, a_dij_1))
            #        if (d_zij - d_zij_1) == 0.:
            #            print("Division by 0 at {}".format(layer))
            #        delta_z = (d_z[i] - d_zij_1) / (d_zij - d_zij_1)
            #        a_d[i] = delta_z * (a_dij - a_dij_1) + a_dij_1
            #        v_zt[i] = delta_z * (v_ztij - v_ztij_1) + v_ztij_1
            #        #delta_z = (d_z[i] - d_zi[j - 1]) / (d_zi[j] - d_zi[j - 1])
            #        #a_d[i] = delta_z * (a_di[j] - a_di[j - 1]) + a_di[j - 1]
            #        #v_zt[i] = delta_z * (v_zti[j] - v_zti[j - 1]) + v_zti[j - 1]
            #    else:
            #        if (d_z[i] > d_zi_d_nn_p1) and (i <= d_n + 1):  # TODO this condition does not depend on j
            #            # print("Outer if at i = {}".format(i))
            #            delta_z = (d_z[i] - d_zi_d_nn) / (d_zi_d_nn_p1 - d_zi_d_nn)
            #            a_d[i] = delta_z * (a_di_d_nn_p1 - a_di_d_nn) + a_di_d_nn
            #            v_zt[i] = delta_z * (v_zti_d_nn_p1 - v_zti_d_nn) + v_zti_d_nn    #for j in range1(2, d_nn + 1):

                #    if (d_z[i] > d_zi[j - 1]) and (d_z[i] <= d_zi[j]):  # TODO add break once it goes through the if

            # Calculate layer volume(m^3)
            #for j in range1(1, d_n):
            #    d_v[j] = v_zt[j + 1] - v_zt[j]
            #    dd_z[j] = d_z[j + 1] - d_z[j]
            d_v[1: d_n + 1]=(shift(v_zt[1:d_n + 2], -1) - v_zt[1:d_n + 2])[0:d_n]
            dd_z[1: d_n + 1]=(shift(d_z[1:d_n + 2], -1) - d_z[1:d_n + 2])[0:d_n]

        # Intitialize layer temperature and total storage
        #if (ti == 1) and (ww == 1):
            #for j in range1(1, d_n):
            #    t_z[j] = t_air
            #    rho_z[j] = den(t_z[j])
            #    m_zo[j] = V_cf * d_v[j] * rho_z[j]
            #    m_zn[j] = m_zo[j]
            t_z[1:d_n+1] = t_air
            rho_z[1:d_n+1] = den(t_z[1:d_n+1])
            m_zo[1:d_n+1] = V_cf * d_v[1:d_n+1] * rho_z[1:d_n+1]
            m_zn[1:d_n+1] = m_zo[1:d_n+1]
            s_tin = v_zt[d_n + 1]
            s_t = s_tin
            m_intial = sum(m_zo)

        
        ########################################################
        # End of the relocated section                         #
        ########################################################


        # ************************************************************************
        # Just print something to keep track of what's happening...
        # the statement below prints the progress through the time steps
        print(ti)
        for ww in range1(1, s_dtime):  # Start calculation for each sub-timestep ***
            # the statemetn below prints progress through sub-time steps
            #print(ti,ww)
            d_n_n = d_n

            # else:
            # Pointless to assign a variable to itself within a loop!!!
            #for j in range1(1, d_n): 
            #    t_z[j] = t_z[j]
            s_t = v_zt[d_n + 1]

            # Allocate layer temperature as old for assigning counter 
            # for averaging sub-timestep result
            #for j in range1(1, d_n):
            t_z_old[:] = t_z[:]

            # Calculation of Surface properties like: albedo, light extinction coefficient, etc
            if (coszen > 0.0):
                alb_s = 0.05 / (0.15 + coszen)
            else:
                alb_s = 0.06

            ##################################################
            # I wonder if all the various sub models for the # 
            # geophysical processes should become their own  #
            # indipendedn functions (mostly for readebility  #
            ##################################################
            
            # as used in Subin et al, 2011 (citing Hakanson, 1995) but modified for actual reservoir depth
            eta = 1.1925 * (d_res) ** (-0.424)
            beta = 0.175
            bias = 1.00

            # Calculation of Surface fluxes and heat source
            # Net shortwave radiation (w/m^2)
            sh_net = max(bias * s_w * (1 - alb_s), 0.)
            t_s = t_z[d_n]  # Initialize surface temperature
            lw_abr = (1. - 0.03) * lw_abs  # longwave radiation (w/m^2)
            es = 4.596 * exp(17.25 * (t_s - 273.15) / t_s)  # in mmHg
            ea = 0.01 * rh * 4.596 * exp(17.25 * (t_air - 273.15) / t_air)  # in mmHg
            lw_ems = 0.97 * st_bl * t_s ** 4  # as used in henderson-sellers, 1984 (w/m^2)
            sn_heat = 1.5701 * u_2 * (t_s - t_air)  # sensible heat (w/m^2)

            # Evaporation calculated as in Wu et al, 2012
            kl = 0.211 + 0.103 * u_2 * F
            evap = max(kl * 133.322368 * (es - ea) / 100., 0.)  # in mm/d; ea and es converted from mmHg to hpa
            if (t_s > t_frz):
                lt_heat = rho_w * evap * le / (86.4e6)  # latent heat (w/m^2)
            else:
                lt_heat = 0.0
            if (in_f == 0.):
                evap = 0.  # Avoid continuous water abstraction if there is no inflow.
                # Assumption is precipitation accounts for evaporation
            # net surface heat flux
            phi_o = sh_net + lw_abr - lt_heat - lw_ems - sn_heat

            # Calculation of reservoir density at depth z and incoming flow
            #for j in range1(1, d_n): #_max):
            #    #if (j <= d_n):
            #    rho_z[j] = den(t_z[j])
            #    #else:
            rho_z[1:d_n + 1] = den(t_z[1:d_n + 1])
            rho_z[d_n+1:] = 0.

            rho_r = den(in_t)

            # Calculation of equivalent evaporated depth (m), and volume(m^3)
            d_evap = evap * dtime / (86.4e6)
            v_evap = max(d_evap * A_cf * a_d[d_n + 1], 0.)
            m_ev = max(v_evap * rho_z[d_n - 1], 0.)

            ########################################################
            # Would be good to try to clarify what all the options #
            # below are meant to do, as of now not very clear...   #
            ########################################################
            
            # Calculation of flow contibution due to inflow/outflow
            if (s_t <= 0.10 * (s_tin + V_df * 1e6)) and (ou_f > in_f):
                flag = 1
            if ((d_res < 5.) and (d_n <= 3) and (ou_f > in_f)):
                flag = 1
            if (s_t <= 0.10 * (s_tin + V_df * 1e6) and (ou_f > in_f)):
                ou_f = in_f  # Avoid extraction of water from reservoir beyond 20% volume of the total storage
            if (s_t >= (s_tin + V_df * 1e6)) and (in_f > ou_f):
                ou_f = in_f  # Avoid extra inflow of water if the reservoir storage exceeded total dam storage (taken to be initial storage)
            if (d_res >= d_ht) and (in_f > ou_f):
                ou_f = in_f  # Avoid reservoir level from exceeding dam height
            if ((d_res < 5.) and (d_n <= 3) and (ou_f > in_f)):
                ou_f = in_f  # Avoid extraction of water from shallow reservoirs beyond 5m depth for numerical stability until reservoir operation is calibrated
            if (flag == 1):
                v_evap = 0.
            if (flag == 1):
                m_ev = 0.

            # And now we load the flow data
            dv_in, dv_ou, dm_in, th_en = flowdist(d_n, in_f, in_t, ou_f, dtime, d_v, v_zt, c_w)

            # Resize layer thickness and numbers based on inflow/outflow 
            # contribution
            
            # **********************************************************
            # I left the if below even though it should have been      #
            # moved out of the internal loop, becuase it uses variable #
            # calculated in this internal loop in the steps above...   #
            ############################################################
            
            # Calculate initial layer and total mass (kg)
            if (ti == 1) and (ww == 1):
                #for j in range1(1, d_n):
                #    m_zo[j] = V_cf * d_v[j] * rho_z[j]
                m_zo[1:d_n+1] = V_cf * d_v[1:d_n+1] * rho_z[1:d_n+1]
                m_intial = sum(m_zo)
            else:
                #for j in range1(1, d_n): #_max):
                #    #if (j <= d_n):
                #    m_zo[j] = m_zn[j]
                #    #else:
                m_zo[1:d_n+1] = m_zn[1:d_n+1]
                m_zo[d_n+1:] = 0.

            #for j in range1(1, d_n): #_max):
            #    #if (j <= d_n):
            #    dm_ou[j] = dv_ou[j] * rho_z[j] * dtime
            #    #else:
            dm_ou[1:d_n+1] = dv_ou[1:d_n+1] * rho_z[1:d_n+1] * dtime
            dm_ou[d_n+1:] = 0.

            # 999
            test999=True
            
            ################ C A U T I O N ###################
            # I'm confident enough that the while structures #
            # below mimic the behavior of the original goto  #
            # statements... however...                       #
            ##################################################
            
            while test999:
                exit_loop = False
                test999 = False
                if d_n > 1:
                    #for j in range1(1, d_n-1):
                    #if (j < d_n - 1): # and (d_n > 1):
                    #    dm_nt[j] = dm_in[j] - dm_ou[j]
                    #    dv_nt[j] = dv_in[j] - dv_ou[j]
                    dm_nt[1:d_n] = dm_in[1:d_n] - dm_ou[1:d_n]
                    dv_nt[1:d_n] = dv_in[1:d_n] - dv_ou[1:d_n]
                    #elif (j == d_n - 1): # and (d_n > 1):
                    dm_nt[d_n - 1] = dm_in[d_n - 1] - dm_ou[d_n - 1] - m_ev
                    dv_nt[d_n - 1] = dv_in[d_n - 1] - dv_ou[d_n - 1] - v_evap
                    #elif (j == d_n): # and (d_n > 1):
                    dm_nt[d_n] = dm_in[d_n] - dm_ou[d_n]
                    dv_nt[d_n] = dv_in[d_n] - dv_ou[d_n]
                elif (d_n == 1):
                    # (j == d_n)
                    dm_nt[d_n] = dm_in[d_n] - dm_ou[d_n] - m_ev
                    dv_nt[d_n] = dv_in[d_n] - dv_ou[d_n] - v_evap

                # Calculate layer mass (kg) and energy (w)
                num_fac = 1.e6
                num_fac1 = 1.e3
                #for j in range1(1, d_n): #_max):
                #    #if (j <= d_n):
                #    m_zn[j] = m_zo[j] + dm_nt[j]
                #    fac_1[j] = V_cf * d_v[j] * rho_z[j] * c_w / dtime
                #    enr_0[j] = t_z[j] * fac_1[j] / num_fac
                #    #else:
                m_zn[1:d_n+1] = m_zo[1:d_n+1] + dm_nt[1:d_n+1]
                fac_1[1:d_n+1] = V_cf * d_v[1:d_n+1] * rho_z[1:d_n+1] * c_w / dtime
                enr_0[1:d_n+1] = t_z[1:d_n+1] * fac_1[1:d_n+1] / num_fac
                m_zn[d_n+1:] = 0. # TODO Why not also fac_1??
                enr_0[d_n+1:] = 0.

                if (ti == 1) and (ww == 1):
                    m_cal = sum(m_zo) + sum(dm_nt)
                    m_mod = sum(m_zo) + sum(dm_nt)
                else:
                    m_cal = m_cal + sum(dm_nt)
                    m_mod = sum(m_zo) + sum(dm_nt)

                d_z[1] = 0.
                a_d[1] = 0.1
                v_zt[1] = 0.1
                
                ######################################################
                # There needs to be a better way of implementing all #
                # the conditions below...                            #
                # We defenitely need to understand what happens here #
                # and then maybe it's worth trying to generate some  #
                # pseudo-code??                                      #
                ######################################################
                
                for i in range1(1, d_n):  # check layers for available volume to satisfy net outflow
                    if (-1.0 * dm_nt[i] > m_zo[i]):
                        # current layer collapses, hence remaining volume taken from next upper layer
                        m_zn[i] = 0.
                        if (i < d_n - 1) and (d_n > 1):
                            m_zn[i + 1] = m_zn[i + 1] - (-1.0 * dm_nt[i] - m_zo[i])
                        elif (i == d_n - 1) and (d_n > 2):
                            # sub-layer collapses, hence remaining mass taken from next lower layer
                            m_zn[i - 1] = m_zn[i - 1] - (-1.0 * dm_nt[i] - m_zo[i])
                            m = i - 1
                            for k in range1(m, d_n):
                                v_zt[k + 1] = v_zt[k] + m_zn[k] / rho_z[k]
                                a_d[k + 1],d_z[k + 1] = LayerChar_v(v_zt[k + 1], dfGeom, d_nn)
                                #for j in range1(2, d_nn + 1):
                                #    if (v_zt[k + 1] > v_zti[j - 1]) and (v_zt[k + 1] <= v_zti[j]):
                                #        delta_z = (d_zi[j] - d_zi[j - 1]) * (v_zt[k + 1] - v_zti[j - 1]) / (
                                #                    v_zti[j] - v_zti[j - 1])
                                #        delta_a = (a_di[j] - a_di[j - 1]) * (v_zt[k + 1] - v_zti[j - 1]) / (
                                #                    v_zti[j] - v_zti[j - 1])
                                #        d_z[k + 1] = d_zi[j - 1] + delta_z
                                #        a_d[k + 1] = a_di[j - 1] + delta_a
                                #    elif (v_zt[k + 1] > v_zti[d_nn + 1]):
                                #        delta_z = (d_zi[d_nn + 1] - d_zi[d_nn]) * (v_zt[k + 1] - v_zti[d_nn]) \
                                #                  / (v_zti[d_nn + 1] - v_zti[d_nn])
                                #        delta_a = (a_di[d_n + 1] - a_di[d_n]) * (v_zt[k + 1] - v_zti[d_nn]) \
                                #                  / (v_zti[d_nn + 1] - v_zti[d_nn])
                                #        d_z[k + 1] = d_zi[d_nn] + delta_z
                                #        a_d[k + 1] = a_di[d_nn] + delta_a
                                d_z[k] = d_z[k + 1]
                                dd_z[k] = d_z[k + 1] - d_z[k]
                        elif ((i == d_n - 1) or (i == d_n)) and (d_n <= 2):  # Top layer collapses, skip outflow
                            dm_ou[i] = 0.
                            exit_loop = True
                            test999 = True
                            break
                            # go to 999

                        v_zt[i + 1] = v_zt[i] + m_zn[i] / rho_z[i]
                        a_d[i + 1], d_z[i + 1] = LayerChar_v(v_zt[i + 1], dfGeom, d_nn)
                        #for j in range1(2, d_nn + 1):
                        #    if (v_zt[i + 1] > v_zti[j - 1]) and (v_zt[i + 1] <= v_zti[j]):
                        #        delta_z = (d_zi[j] - d_zi[j - 1]) * (v_zt[i + 1] - v_zti[j - 1]) / (v_zti[j] - v_zti[j - 1])
                        #        delta_a = (a_di[j] - a_di[j - 1]) * (v_zt[i + 1] - v_zti[j - 1]) / (v_zti[j] - v_zti[j - 1])
                        #        d_z[i + 1] = d_zi[j - 1] + delta_z
                        #        a_d[i + 1] = a_di[j - 1] + delta_a
                        #    elif (v_zt[i + 1] > v_zti[d_nn + 1]):
                        #        delta_z = (d_zi[d_nn + 1] - d_zi[d_nn]) * (v_zt[i + 1] - v_zti[d_nn]) / (
                        #                    v_zti[d_nn + 1] - v_zti[d_nn])
                        #        delta_a = (a_di[d_n + 1] - a_di[d_n]) * (v_zt[i + 1] - v_zti[d_nn]) / (
                        #                    v_zti[d_nn + 1] - v_zti[d_nn])
                        #        d_z[i + 1] = d_zi[d_nn] + delta_z
                        #        a_d[i + 1] = a_di[d_nn] + delta_a
                        d_z[i] = d_z[i + 1]
                        dd_z[i] = d_z[i + 1] - d_z[i]
                    else:  # enough volume, layers don't collapses
                        v_zt[i + 1] = v_zt[i] + m_zn[i] / rho_z[i]
                        a_d[i + 1], d_z[i + 1] = LayerChar_v(v_zt[i + 1], dfGeom, d_nn)
                        #for j in range1(2, d_nn + 1):
                        #    if (v_zt[i + 1] > v_zti[j - 1]) and (v_zt[i + 1] <= v_zti[j]):
                        #        delta_z = (d_zi[j] - d_zi[j - 1]) * (v_zt[i + 1] - v_zti[j - 1]) / (v_zti[j] - v_zti[j - 1])
                        #        delta_a = (a_di[j] - a_di[j - 1]) * (v_zt[i + 1] - v_zti[j - 1]) / (v_zti[j] - v_zti[j - 1])
                        #        d_z[i + 1] = d_zi[j - 1] + delta_z
                        #        a_d[i + 1] = a_di[j - 1] + delta_a
                        #    elif (v_zt[i + 1] > v_zti[d_nn + 1]):
                        #        delta_z = (d_zi[d_nn + 1] - d_zi[d_nn]) * (v_zt[i + 1] - v_zti[d_nn]) / (
                        #                    v_zti[d_nn + 1] - v_zti[d_nn])
                        #        delta_a = (a_di[d_n + 1] - a_di[d_n]) * (v_zt[i + 1] - v_zti[d_nn]) / (
                        #                    v_zti[d_nn + 1] - v_zti[d_nn])
                        #        d_z[i + 1] = d_zi[d_nn] + delta_z
                        #        a_d[i + 1] = a_di[d_nn] + delta_a
                        dd_z[i] = d_z[i + 1] - d_z[i]
#                if not exit_loop:
#                    test999=False
            # Recalculate layer thickness and volume
            #for i in range1(1, d_n): Merged with the next loop down
            #    d_v[i] = m_zn[i] / rho_z[i]

            # Recalculate reservoir depth
            d_res = 0.
            for i in range1(1, d_n):
                d_v[i] = m_zn[i] / rho_z[i]
                d_res = d_res + dd_z[i]

            ######################################################
            # Similar consideration as above...                  #
            # We defenitely need to understand what happens here #
            # and then maybe it's worth trying to generate some  #
            # pseudo-code??                                      #
            ######################################################
        
            # Check if layers are too small
            #   71
            test71 = True
            while test71 and (d_n >= 1):
                test71 = False
                for i in range1(1, (d_n - 1)):
                    if (dd_z[i] < ddz_min):
                        test71 = True
                        # go to 75
                        # Identify lower (small and to be merged) and upper (larger) layer
                        #   75
                        if (i < d_n - 1):  # TODO rework the condition. Set default to m = i + 1 and
                            # if i = d_n-1 set to m = i - 1
                            m = i + 1
                        else:
                            m = i - 1  # Avoid merging to top layer

                        ta = t_z[i]
                        tb = t_z[m]
                        dd_za = dd_z[i]
                        dd_zb = dd_z[m]
                        m_a = m_zn[i]
                        m_b = m_zn[m]
                        e_a = enr_0[i]
                        e_b = enr_0[m]
                        d_va = d_v[i]
                        d_vb = d_v[m]
                        dv_oua = dv_ou[i]
                        dv_oub = dv_ou[m]
                        dv_ina = dv_in[i]
                        dv_inb = dv_in[m]

                        # Merge layers
                        if (d_va + d_vb > 0.):
                            t_z[i] = (d_va * ta + d_vb * tb) / (d_va + d_vb)

                        # Adjust new layer volume, thickness, mass and inflow/outflow
                        dd_z[i] = dd_za + dd_zb
                        d_v[i] = d_va + d_vb
                        m_zn[i] = m_a + m_b
                        dv_ou[i] = dv_oua + dv_oub
                        dv_in[i] = dv_ina + dv_inb
                        enr_0[i] = e_a + e_b

                        # Re-number layers before collapse
                        for j in range1(i, d_n_max - 1):
                            if (j == i) and (i < (d_n - 1)):
                                t_z[j] = t_z[i]
                                dv_ou[j] = dv_ou[i]
                                dv_in[j] = dv_in[i]
                                dd_z[j] = dd_z[i]
                                d_v[j] = d_v[i]
                                m_zn[j] = m_zn[i]
                                enr_0[j] = enr_0[i]
                                d_z[j + 1] = d_z[i + 2]
                                v_zt[j + 1] = v_zt[i + 2]
                                a_d[j + 1] = a_d[i + 2]
                            elif (j < d_n) and (i < (d_n - 1)):
                                t_z[j] = t_z[j + 1]
                                dv_ou[j] = dv_ou[j + 1]
                                dv_in[j] = dv_in[j + 1]
                                dd_z[j] = dd_z[j + 1]
                                d_v[j] = d_v[j + 1]
                                m_zn[j] = m_zn[j + 1]
                                enr_0[j] = enr_0[j + 1]
                                d_z[j + 1] = d_z[j + 2]
                                v_zt[j + 1] = v_zt[j + 2]
                                a_d[j + 1] = a_d[j + 2]
                            elif (j == i) and (i == (d_n - 1)):
                                t_z[j - 1] = t_z[i]
                                dv_ou[j - 1] = dv_ou[i]
                                dv_in[j - 1] = dv_in[i]
                                dd_z[j - 1] = dd_z[i]
                                d_v[j - 1] = d_v[i]
                                m_zn[j - 1] = m_zn[i]
                                enr_0[j - 1] = enr_0[i]
                                d_z[j] = d_z[j + 1]
                                v_zt[j] = v_zt[j + 1]
                                a_d[j] = a_d[j + 1]
                                t_z[d_n - 1] = t_z[d_n]
                                dv_ou[d_n - 1] = dv_ou[d_n]
                                dv_in[d_n - 1] = dv_in[d_n]
                                dd_z[d_n - 1] = dd_z[d_n]
                                d_v[d_n - 1] = d_v[d_n]
                                m_zn[d_n - 1] = m_zn[d_n]
                                enr_0[d_n - 1] = enr_0[d_n]
                                d_z[d_n] = d_z[d_n + 1]
                                v_zt[d_n] = v_zt[d_n + 1]
                                a_d[d_n] = a_d[d_n + 1]
                            elif (j >= d_n):
                                t_z[j] = 0.
                                dv_ou[j] = 0.
                                dv_in[j] = 0.
                                dd_z[j] = 0.
                                d_v[j] = 0.
                                m_zn[j] = 0.
                                enr_0[j] = 0.
                                d_z[j + 1] = 0.
                                v_zt[j + 1] = 0.
                                a_d[j + 1] = 0.
                        d_n = d_n - 1
                        if (d_n < 1):
                            break4099 = True
                        break
#                    else:
#                        test71 = False

                # go to 4099   # TODO need to implement the logic for the goto 4099
                # (also called later in the code)
                # it seems to require "breaking" from 3 loops!!!
            # go to 71
            # go to 91
            if break4099:
                break

            ######################################################
            # Similar consideration as above...                  #
            # We defenitely need to understand what happens here #
            # and then maybe it's worth trying to generate some  #
            # pseudo-code??                                      #
            ######################################################

            # Check if layers are too big
            #   91
            test91 = True
            while test91:
                test91 = False
                for i in range1(1, d_n):
                    if (dd_z[i] >= ddz_max):  # go to 95
                        # Calculate layer geometric properties to be halved
                        test91 = True
                        #   95
                        dd_zab = 0.5 * dd_z[i]
                        m_ab = 0.5 * m_zn[i]
                        e_ab = 0.5 * enr_0[i]
                        d_vab = d_v[i]
                        tab = t_z[i]
                        dv_ouab = dv_ou[i]
                        dv_inab = dv_in[i]

                        # Re-number layers before dividing layer
                        d_z[d_n + 2] = d_z[d_n + 1]
                        v_zt[d_n + 2] = v_zt[d_n + 1]
                        a_d[d_n + 2] = a_d[d_n + 1]
                        m = i + 1
                        for j in range1(m, d_n):
                            k = d_n - j + i + 1
                            t_z[k + 1] = t_z[k]
                            dv_ou[k + 1] = dv_ou[k]
                            dv_in[k + 1] = dv_in[k]
                            d_z[k + 1] = d_z[k]
                            a_d[k + 1] = a_d[k]
                            v_zt[k + 1] = v_zt[k]
                            d_v[k + 1] = d_v[k]
                            dd_z[k + 1] = dd_z[k]
                            m_zn[k + 1] = m_zn[k]
                            enr_0[k + 1] = enr_0[k]

                        # Divide layer in half and calculate corresponding properties
                        dd_z[i] = dd_zab
                        dd_z[i + 1] = dd_zab
                        d_z[i + 1] = d_z[i] + dd_z[i]
                        a_d[i + 1], v_zt[i + 1] = LayerChar_d(d_z[i + 1], dfGeom, d_nn)
                        #for j in range1(2, d_nn + 1):
                        #    if (d_z[i + 1] > d_zi[j - 1]) and (d_z[i + 1] <= d_zi[j]):
                        #        delta_z = (d_z[i + 1] - d_zi[j - 1]) / (d_zi[j] - d_zi[j - 1])
                        #        a_d[i + 1] = delta_z * (a_di[j] - a_di[j - 1]) + a_di[j - 1]
                        #        v_zt[i + 1] = delta_z * (v_zti[j] - v_zti[j - 1]) + v_zti[j - 1]
                        #    elif (d_z[i + 1] > d_zi[d_nn + 1]):
                        #        delta_z = (d_z[i + 1] - d_zi[d_nn]) / (d_zi[d_nn + 1] - d_zi[d_nn])
                        #        a_d[i + 1] = delta_z * (a_di[d_nn + 1] - a_di[d_nn]) + a_di[d_nn]
                        #        v_zt[i + 1] = delta_z * (v_zti[d_nn + 1] - v_zti[d_nn]) + v_zti[d_nn]
                        d_v[i + 1] = d_vab - (v_zt[i + 1] - v_zt[i])
                        d_v[i] = v_zt[i + 1] - v_zt[i]
                        dv_ou[i + 1] = d_v[i + 1] * dv_ouab / (d_v[i] + d_v[i + 1])
                        dv_ou[i] = d_v[i] * dv_ouab / (d_v[i] + d_v[i + 1])
                        t_z[i] = tab
                        t_z[i + 1] = tab
                        m_zn[i] = m_ab
                        m_zn[i + 1] = m_ab
                        enr_0[i] = e_ab
                        enr_0[i + 1] = e_ab
                        dv_in[i + 1] = d_v[i + 1] * dv_inab / (d_v[i] + d_v[i + 1])
                        dv_in[i] = d_v[i] * dv_inab / (d_v[i] + d_v[i + 1])
                        d_n = d_n + 1
                        break
#                    else:
#                        test91 = False
                        # go to 91

            # go to 99

            # 99		continue
            # Recalculate density after layer change
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):  # then
            #        rho_z[j] = den(t_z[j])
            #    else:
            #        rho_z[j] = 0.
            rho_z[1:d_n+1] = den(t_z[1:d_n+1])
            rho_z[d_n+1:] = 0.

            s_t = v_zt[d_n + 1]
            if (s_t <= 0.0050 * (s_tin + V_df * 1e6)) or (d_res <= 1.):
                break4099 = True
                # go to 4099
            if (d_n == 1) and (abs(t_z[d_n]) >= 450.):
                break4099 = True
                # go to 4099
            if (sum(m_zn) <= 0.):
                break4099 = True
                # go to 4099
            if (d_n >= d_n_max):
                break4099 = True
                # go to 4099

            if break4099:
                break

            # Calculate layer internal energy (w) due to inflow/outflow
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):
            #        enr_in[j] = dv_in[j] * in_t * rho_r * c_w / num_fac  # Energy from inflow
            #        enr_ou[j] = dv_ou[j] * t_z[j] * rho_z[j] * c_w / num_fac  # Energy loss due to outflow
            #    else:
            #        enr_in[j] = 0.
            #        enr_ou[j] = 0.
            enr_in[1:d_n+1] = dv_in[1:d_n+1] * in_t * rho_r * c_w / num_fac  # Energy from inflow
            enr_ou[1:d_n+1] = dv_ou[1:d_n+1] * t_z[1:d_n+1] * rho_z[j] * c_w / num_fac  # Energy loss due to outflow
            enr_in[d_n+1:] = 0.
            enr_ou[d_n+1:] = 0.

            # Calculate layer energy (w)
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):
            #        enr_1[j] = enr_0[j] + enr_in[j] - enr_ou[j]
            #    else:
            #        enr_1[j] = 0.
            enr_1[1:d_n+1] = enr_0[1:d_n+1] + enr_in[1:d_n+1] - enr_ou[1:d_n+1]
            enr_1[d_n+1:] = 0.

            # Adjust layer temperature for mixing due to inflow/outflow
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):
            #        t_z[j] = enr_1[j] * num_fac * dtime / (m_zn[j] * c_w)
            #    else:
            #        t_z[j] = 0.
            t_z[1:d_n+1] = enr_1[1:d_n+1] * num_fac * dtime / (m_zn[1:d_n+1] * c_w)
            t_z[d_n+1:] = 0.

            # Check energy balance (w) after advective mixing
            enr_err1 = (sum(enr_1) - (sum(enr_0) + sum(enr_in) - sum(enr_ou))) * num_fac

            # ******************************************************************************
            # Calculate solar energy absorbed at each layer
            #for j in range1(1, d_n_max):
            #    phi_x[j] = 0.
            #    phi_z[j] = 0.
            phi_x[:] = 0.
            phi_z[:] = 0.

            if (sh_net > 0.):
                k = 0
                top_d = d_res - d_z[d_n - k]
                while top_d < 0.61:
                    # 600
                    k = k + 1
                    top_d = d_res - d_z[d_n - k]
                # if(top_d < 0.61) then
                #    k=k+1
                #    go to 600
                # else
                v_mix = V_cf * (v_zt[d_n + 1] - v_zt[d_n - k])
                # Solar radiation energy absorbed at the mixed zone
                sh_mix = sh_net * A_cf * (a_d[d_n + 1] - (1. - beta) * a_d[d_n - k])
                j = d_n - k
                for i in range1(j, d_n):
                    phi_z[i] = sh_mix * V_cf * d_v[i] / v_mix
                phi_x[d_n + 1] = sh_net
                phi_x[d_n - k] = (1. - beta) * sh_net
                if (k > 0):
                    for i in range1(1, k):
                        ii = d_n - i + 1
                        phi_x[ii] = (A_cf * a_d[ii + 1] * phi_x[ii + 1] - phi_z[ii]) / (A_cf * a_d[ii])
                # Solar radiation energy absorbed at sub layers
                l = d_n - k - 1
                for j in range1(1, l):
                    i = (d_n - k) - j + 1
                    phi_x[i - 1] = phi_x[i] * exp(-1 * eta * (d_z[i] - d_z[i - 1]))

                # Solar radiation energy absorbed in each layer
                j = d_n - k - 1
                for i in range1(1, j):
                    phi_z[i] = A_cf * (a_d[i + 1] * phi_x[i + 1] - a_d[i] * phi_x[i])
            else:
                #for j in range1(1, d_n):
                #    phi_z[j] = 0.
                phi_z[1:d_n+1] = 0.

            # *************************************************************************************
            # Calculation of effective diffusion coefficient
            if (u_2 >= 15.):
                c_d = 2.6e-3
            else:
                c_d = 5.0e-4 * sqrt(u_2)
            tau = rho_a * c_d * u_2 ** 2.  # Shear stress at surface
            s_vel = sqrt(tau / rho_w)  # Shear velocity at surface
            k_ew = tau * s_vel * A_cf * a_d[d_n + 1] * dtime  # Wind driven kinetic energy at surface
            dis_w = k_ew / (rho_w * V_cf * v_zt[d_n + 1] * dtime)  # rate of dissipation-wind
            cfw = 1.0e-02
            cfa = 1.0e-05
            k_m = 0.57 / (c_w * rho_w)  # molecular diffusivity
            df_eff[1] = 0.  # bottom interface
            df_eff[d_n + 1] = 0.  # air interface
            for j in range1(2, d_n):
                q_adv[j] = max((dv_in[j] + dv_ou[j]), 0.)
                k_ad[j] = 0.5 * rho_w * q_adv[j] * dtime * (
                            q_adv[j] / (M_W * dd_z[j])) ** 2.  # Advection driven kinetic energy
                dis_ad[j] = k_ad[j] / (rho_w * V_cf * v_zt[j] * dtime)  # rate of dissipation-inflow/outflow

                # Calculate Richardson number
                drhodz[j] = (rho_z[j - 1] - rho_z[j]) / 0.5 * (dd_z[j] + dd_z[j - 1])
                bv_f = max((grav / rho_w) * drhodz[j], 0.)
                if (s_vel <= 0.):  # TODO seems a pointless if statement
                    ri = 0.
                ri = bv_f / ((s_vel / (0.4 * d_z[j])) ** 2.)

                # Calculate Froude number
                l_vel = q_adv[j] * M_L / (A_cf * a_d[j] * dd_z[j])
                if (q_adv[j] <= 0.) or (drhodz[j] <= 0.):  # TODO seems another pointless if statement
                    Fr[j] = 0.
                if rho_w == 0 or l_vel == 0:
                    Fr[j]=0 # TODO check if it should be nan...
                else:
                    Fr[j] = (grav * dd_z[j] * drhodz[j] / rho_w) / l_vel ** 2.

                # Calculate diffusion coefficients
                test=dtime ** 2. * ((cfw * dis_w / (1 + ri)) + (0.5 * cfa * (dis_ad[j] + dis_ad[j - 1]) / (1 + Fr[j])))
                df_eff[j] = min(max(test, k_m), 5.56e-03)

            # *****************************************************************
            # Calculate matrix elements
            #for j in range1(1, d_n_max):
            #    a[j] = 0.
            #    b[j] = 0.
            #    c[j] = 0.
            #    r[j] = 0.
            #    # AX[j] = 0.
            #    # BX[j] = 0.
            #    # CX[j] = 0.
            #    # DX[j] = 0.
            #    FX[j] = 0.
            a[:] = 0.
            b[:] = 0.
            c[:] = 0.
            r[:] = 0.
            FX[:] = 0.
                
            for j in range1(1, d_n):  # TODO simplify all the if conditions removing common statements
                if (j == 1) and (d_n > 1):
                    m1[j] = 2 * dtime / (0.5 * A_cf * (a_d[j] + a_d[j + 1]) * dd_z[j])
                    m2[j] = m1[j] * A_cf * a_d[j + 1] * df_eff[j + 1] / (dd_z[j] + dd_z[j + 1])
                    m3[j] = 0.
                    FX[j] = dtime * phi_z[j] / (V_cf * d_v[j] * c_w * rho_z[j])
                    a[j] = -1. * m2[j]
                    b[j] = 1. + m2[j] + m3[j]
                    c[j] = 0.
                    r[j] = t_z[j] + FX[j]  # + dt_in(j) - dt_ou(j)! bottom boundary condition
                elif (j <= d_n - 1) and (d_n > 2):
                    m1[j] = 2 * dtime / (0.5 * A_cf * (a_d[j] + a_d[j + 1]) * dd_z[j])
                    m2[j] = m1[j] * A_cf * a_d[j + 1] * df_eff[j + 1] / (dd_z[j] + dd_z[j + 1])
                    m3[j] = m1[j] * A_cf * a_d[j] * df_eff[j] / (dd_z[j] + dd_z[j - 1])
                    FX[j] = dtime * phi_z[j] / (V_cf * d_v[j] * c_w * rho_z[j])
                    a[j] = -1. * m2[j]
                    b[j] = 1. + m2[j] + m3[j]
                    c[j] = -1 * m3[j]
                    r[j] = t_z[j] + FX[j]
                elif (j == d_n):  # top layer
                    m1[j] = 2 * dtime / (0.5 * A_cf * (a_d[j] + a_d[j + 1]) * dd_z[j])
                    m2[j] = 0.
                    m3[j] = m1[j] * A_cf * a_d[j] * df_eff[j] / (dd_z[j] + dd_z[j - 1])
                    FX[j] = dtime * ((phi_o - sh_net) * A_cf * a_d[d_n + 1] + phi_z[j]) / (
                                V_cf * d_v[d_n] * c_w * rho_z[j])
                    a[j] = 0.
                    b[j] = 1. + m2[j] + m3[j]
                    c[j] = -1. * m3[j]
                    r[j] = t_z[j] + FX[j]  # top boundary condition

            # Solve for temperature --- (a,b,c,r,u,n) u is output
            t_tmp = solve(a, b, c, r, t_z, d_n)
            t_z[1:d_n+1] = t_tmp[1:d_n+1]

            # Avoid Numerical instability for multiple reservoir runs
            # for j in range1(1, d_n):
            if (sum(np.isnan(t_z))):
                print('check reservoir data')
                break4099 = True
                break
                # go to 4099

            # ***********************************************************************************************
            # Solve convective mixing
            # Recalculate layer density
            #for j in range1(1, d_n):
            #    rho_z[j] = den(t_z[j])
            rho_z[1:d_n+1] = den(t_z[1:d_n+1])
            enr_bc = 0.
            enr_ac = 0.

            # Check if instability exists
            k = 1

            # 501   continue
            while k < d_n:
                # if(k >= d_n) go to 560
                while (rho_z[k] >= rho_z[k + 1]):
                    k = k + 1
                    if k==d_n:
                        break
                # if(rho_z(k) < rho_z(k+1)) go to 510
                # k=k+1
                # go to 501
                # 510   continue

                # Start mixing layers
                mixlow = k
                mixtop = mixlow
                sumvol = V_cf * d_v[mixtop]
                summas = m_zn[mixtop]
                sumenr = enr_2[mixtop]
                tsum = t_z[mixtop] * m_zn[mixtop]
                # 520   continue
                test520 = True
                while test520:  # (denmix >= rho_z[mixtop + 1]):
                    mixtop = mixtop + 1
                    vlmxtp = V_cf * d_v[mixtop]
                    msmxtp = m_zn[mixtop]
                    enmxtp = enr_2[mixtop]
                    sumvol = sumvol + vlmxtp
                    summas = summas + msmxtp
                    sumenr = sumenr + enmxtp
                    tsum = tsum + t_z[mixtop] * msmxtp
                    tmix = tsum / summas

                    # Calculate density of mixed layer
                    denmix = den(tmix)
                    if (mixtop == d_n):
                        break  # go to 540
                    if (denmix >= rho_z[mixtop + 1]):
                        test520 = False  # go to 520
                # 540   continue
                # if(mixlow <= 1) go to 550

                # Check if instability exists below mixed layer	and mix layers
                # if(rho_z(mixlow-1) >= denmix) go to 550
                while (mixlow > 1) and (rho_z[mixlow - 1] < denmix):
                    mixlow = mixlow - 1

                    # Calculate temperature of mixed layer
                    vlmxlw = V_cf * d_v[mixlow]
                    msmxlw = m_zn[mixlow]
                    enmxlw = enr_2[mixlow]
                    sumvol = sumvol + vlmxlw
                    summas = summas + msmxlw
                    sumenr = sumenr + enmxlw
                    tsum = tsum + t_z[mixlow] * msmxlw
                    tmix = tsum / summas

                    # Calculate density of mixed layer
                    denmix = den(tmix)
                # go to 540 --> while (mixlow > 1) and (rho_z[mixlow - 1] < denmix)

                # 550   continue

                # Set new layer temperature and density
                #for j in range1(mixlow, mixtop):
                #    rho_z[j] = denmix
                #    t_z[j] = tmix
                rho_z[mixlow:mixtop+1] = denmix
                t_z[mixlow:mixtop+1] = tmix

                # Calculate sum of layer energy in the mixing layer before mixing
                for j in range1(mixlow, mixtop):
                    enr_ac = enr_ac + t_z[j] * m_zn[j] * c_w / (dtime * num_fac)
                k = mixtop
                # go to 501 --> while k < d_n:

            # 560    continue

            # Recalculate layer final internal energy (w) after convective mixing
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):
            #        enr_2c[j] = t_z[j] * m_zn[j] * c_w / (dtime * num_fac)
            #    else:
            #        enr_2c[j] = 0.
            enr_2c[1:d_n+1] = t_z[1:d_n+1] * m_zn[1:d_n+1] * c_w / (dtime * num_fac)
            enr_2c[d_n+1:] = 0.

            # Check energy balance (w) after convective mixing
            # TODO enr_phi does not seem to be initialized anywhere so I'm commenting
            #      it out
            # enr_err2c = sum(enr_2c) - (sum(enr_1) + enr_phi)
            enr_err2c = sum(enr_2c) - sum(enr_1)
            enr_err2c = enr_err2c * num_fac  # /a_d(d_n+1)

            # *********************************************************************************
            # Calculate count for sub-timestep averaging
            for j in range1(1, d_n_max):
                if (t_z_old[j] != 0.) and (t_z[j] != 0.):
                    cntr1[j] = cntr1[j] + 1
                    cntr2[j] = 0
                else:
                    cntr1[j] = 0
                    cntr2[j] = cntr1[j] + 1
                cntr[j] = cntr1[j] + cntr2[j]

            # Cummulative sub-timestep fluxes
            sh_neta = sh_neta + sh_net  # * a_d(d_n+1)
            lw_absa = lw_absa + lw_abr
            lt_heata = lt_heata + lt_heat
            lw_emsa = lw_emsa + lw_ems
            sn_heata = sn_heata + sn_heat
            phi_oa = phi_oa + phi_o  # *a_d(d_n+1)

            m_in_sub[j] = 0.
            m_ou_sub[j] = 0.
            d_res_sub = d_res_sub + d_res
            #for j in range1(1, d_n_max):  # TODO WTF is this loop for??
            #    if (j <= d_n):
            #        phi_z[j] = phi_z[j]
            #    else:
            #        phi_z[j] = 0.
            phi_z[d_n+1:] = 0.
            # Calculate layer depth for profile plot(minimum at the top)
            if (d_n >= 2):
                d_zs[1] = d_res - 0.5 * dd_z[2]
                for j in range1(2, d_n): # _max):
                    #if (j <= d_n):
                    d_zs[j] = d_zs[j - 1] - 0.5 * dd_z[j - 1] - 0.5 * dd_z[j]
                    #else:
                d_zs[d_n+1:] = 0.
                d_zs[d_n] = 0.
            else:
                d_zs[1] = d_res

            # Sum sub-timestep variables
            #for j in range1(1, d_n_max):
            #    t_zsub[j] = t_zsub[j] + t_z[j]
            #    # phi_zsub[j] = phi_zsub[j] + phi_z[j]
            #    d_zsb[j] = d_zsb[j] + d_zs[j]
            t_zsub[1:] = t_zsub[1:] + t_z[1:]
            d_zsb[1:] = d_zsb[1:] + d_zs[1:]
            st_sub = st_sub + s_t

        # end do # TODO here is the first internal loop for the goto 4099
        if break4099:
            break

        d_res = d_res_sub / s_dtime

        if (d_n_n == d_n):
            #for j in range1(1, d_n_max):
            #    t_zs[j] = t_zsub[j] / cntr[j]
            #    # phi_z[j] = phi_zsub[j] / cntr[j]
            #    d_zsb[j] = d_zsb[j] / cntr[j]
            t_zs[1:] = t_zsub[1:] / cntr[1:]
            d_zsb[1:] = d_zsb[1:] / cntr[1:]
        else:
            #for j in range1(1, d_n_max):
            #    if (j <= d_n):
            #        t_zs[j] = t_zsub[j] / cntr[j]
            #        # phi_z[j] = phi_zsub[j] / cntr[j]
            #        d_zsb[j] = d_zsb[j] / cntr[j]
            #    else:
            #        t_zs[j] = 0.
            #        # phi_z[j] = 0.
            #        d_zsb[j] = 0.
            t_zs[1:d_n+1] = t_zsub[1:d_n+1] / cntr[1:d_n+1]
            d_zsb[1:d_n+1] = d_zsb[1:d_n+1] / cntr[1:d_n+1]
            t_zs[d_n+1:] = 0.
            d_zsb[d_n+1:] = 0.

        #for j in range1(d_n_max, 1, -1):  # Reverse layer order so that first layer is at the top
        #    k = d_n - d_n_max + j
        #    if (k >= 1):
        #        t_zf[j] = t_zs[k]
        #        # phi_zf[j] = phi_z[k]
        #        d_zf[j] = d_zsb[k]
        #    else:
        #        t_zf[j] = 0.
        #        # phi_zf[j] = 0.
        #        d_zf[j] = 0.

        # Check if we need to add a column to the output dataframes
        if d_n > len(dfStrat.columns):
            dfDepth["Top-{}".format(d_n-1)]=0
            dfStrat["T_K_{}".format(d_n-1)]=0

        depthRecord = [0] * len(dfDepth.columns)
        depthRecord[0:(5+d_n)] = [ti, d_n, in_f, ou_f, d_res] + list(np.flip(d_zsb[1:d_n + 1]))
        stratRecord = [0] * len(dfStrat.columns)
        stratRecord[0:d_n] = list(np.flip(t_zs[1:d_n+1]))

        dfDepth.loc[dfDepth.index==ti]=depthRecord
        dfStrat.loc[dfStrat.index==ti]=stratRecord

        # do j = 1,d_n_max
        # if (isnan(t_zf(j)))t_zf(j)=-999. ! Avoid nan in temp file
        # end do
        # TODO Save results
        # 15, file='data/outputs/stratification/'//rsrv//'' # open stratification result file
        # 199, file='data/outputs/depth/'//energy//rsrv//'' # open depth result file
        # write(15,40225),t_zf!
        # write(199,50225),ti,d_n,in_f,ou_f,d_res,d_zf !write averaged reservoir depth, inflow/outflow

        # 40225 format(50f9.1)
        # 50225 format(i6,1X,i2,1X,f9.2,1X,f9.2,1X,f9.3,1x,50f9.1)
    print('saving files')
    dfStrat.to_csv(os.path.join(outPath,"outputs","stratification",fileIn.replace(".txt",".csv")))
    dfDepth.to_csv(os.path.join(outPath,"outputs","depth",fileIn.replace(".txt",".csv")))
    # end do

    # 4099 continue

# end do

# end program ResStrat